# Q learning for connect four
In this document the use of the reinforment leanring _Q learning_ will be tested. an algrithm for connect four will be trained. as an opponent the connect four AI will be used.

## Algrithmn
Q-learning is a reinforcement learning technique used in machine learning. The technique does not require a model of the environment. Q-learning can handle problems with stochastic transitions and rewards, without requiring adaptations.[1]
Q learning builds a Q matrix of states, transitions (actions) and the evaluation of these.

In this example the scores the node (status) and the game actions (inserting a game chip in the player's color into a column) the transitions. If a player has won in a score, the final node is reached. Formally this can be disengaged like next (with a learnrate $\alpha$ and a discovery rate $\gamma$):

$
Q(s_{t},a_{t}) = (1-\alpha) \cdot Q(s_{t},a_{t}) + \alpha \bigg( \gamma \underbrace{\max_{i} Q(s_{t+1}, a_{i})}_{\rm estimate~of~optimal~future~value} \bigg)
$


The algrithm now adjusts the transitions during teaching, so that the maximum path to a final state is reached. During learning the conditions are visited and examined randomly.

The final algrithm selects the action with the highest transition value from the Q matrix.

### Adaptions to connect four 
For the problem examined here, some adjustments have to be made. Instead, the Q matrix is updated and played against another AI over and over again. However, due to the large number of stands, not a small part of the nodes are visited. Therefore, at the end of a game, all nodes in the Q matrix are updated.

It is also problematic to allocate a matrix for all possible game states. Therefore, a dictionary was used here. To model the state of the game the Horner's method[2] is used. If $b_{i}$ is the value of the playing field $i$ with red equals 1, white equals 2 and empty equals 0, this can be formulated as follows:

$\sum_{i}^{42} b_{i} 3^{i}$

### Requirements
For this example the ConnectFourAILib is used. You can download it [here](https://github.com/jeroen-andress/ConnectFourAILib). It must be compiled and installed for Python3.

## Summary
* The algrithm always selects only the action with the highest rating. therefore, however, we do not consider the advantage of against. if, for example, the opponent has won in a turn, a turn is still chosen that leads to a victory.

* Due to the very large search space (small compared to chess or go) many training games are necessary. 

* The arithmetic is also able to teach during test phase.

## Author
Jeroen Andreß, M.Sc. Professional software developer.

## References
* [1]: https://en.wikipedia.org/wiki/Q-learning
* [2]: https://en.wikipedia.org/wiki/Horner%27s_method
* [Reinforcement Learning - A Simple Python Example and a Step Closer to AI with Assisted Q-Learning](http://amunategui.github.io/reinforcement-learning)
* [A Painless Q-Learning Tutorial](http://mnemstudio.org/path-finding-q-learning-tutorial.htm)


In [1]:
from ConnectFourAI import ConnectFourPlayboard
from ConnectFourAI import ConnectFourPlayerMinimaxAlphaBeta
from ConnectFourAI import ConnectFourPlayerMinimax
from ConnectFourAI import ConnectFourPlayerInterface

import numpy as np

import pylab as plt
%matplotlib inline  

from IPython import display
import time

### Helper functions
The following method is used to display the board.

In [2]:
from ipythonblocks import BlockGrid, colors, fui_colors

def show_playboard(playboard):
    grid = BlockGrid(playboard.GetColumns(), playboard.GetRows(), fill=fui_colors.MidnightBlue)
    
    for r in range(playboard.GetRows()):
        for c in range(playboard.GetColumns()):
            if playboard.GetField(r, c) == playboard.WHITE:
                grid[r, c] = fui_colors.Clouds
            if playboard.GetField(r, c) == playboard.RED:
                grid[r, c] = fui_colors.Pomegranate

    grid.show()

Functions to detect actions on playboard and to en/decode playboard from/to a number represtion.

In [3]:
def available_actions(playboard):
    return [i for i in range(playboard.GetColumns()) if playboard.IsInsertAble(i)]

def encode_playboard(playboard):
    result = 0
    for i in range(playboard.GetRows()):
        for j in range(playboard.GetColumns()):
            r = playboard.GetRows() - i - 1
            c = j 

            result += playboard.GetField(r, c) * 3 ** (i * playboard.GetColumns() + c)

    return result

def decode_playboard(v):
    # Only for debugging
    result = ConnectFourPlayboard()

    for i in range(result.GetRows()):
        for c in range(result.GetColumns()):
            if (v % 3 == ConnectFourPlayboard.RED):
                result.MoveRed(c)
            elif (v % 3 == ConnectFourPlayboard.WHITE):
                result.MoveWhite(c)
            v = int(v / 3)

    return result

In [4]:
class ConnectFourQLearningPlayer(ConnectFourPlayerInterface):
    def __init__(self, player):
        # public attribute
        self.playboard = None
        
        # protected
        self._player = player
        self._Q = {}

    # Impl interface
    def GetPlayer(self):
        return self._player
    
    def MakeMove(self):
        assert self.playboard is not None
        action, v = self._next_action(self.playboard)
        self.playboard.Move(int(action), self._player)
       
        return action
     
    # Internal functions
    def _sample_next_action(self, playboard):
        return int(np.random.choice(available_actions(playboard)))
    
    def _next_action(self, playboard):
        encoded = encode_playboard(playboard)
        
        if encoded in self._Q:
            actions = self._Q[encoded]
            return (np.argmax(actions), np.max(actions))
        else:
            actions = available_actions(playboard)
            return (np.random.choice(actions) if len(actions) > 0 else 0, -1)
    
    def _copy_and_insert_into_playboard(self, playboard, column, player):
        result = ConnectFourPlayboard(playboard)
        result.Move(column, player)
    
        return result
    
    def _get_against_players(self, player):
        return ConnectFourPlayboard.RED if player == ConnectFourPlayboard.WHITE else ConnectFourPlayboard.RED

    def _R(self, playboard, player):
        if playboard.GetWinner() == player:
            return 1.0
        elif playboard.GetWinner() == ConnectFourPlayboard.NONE:
            return 0.0
        else:
            return -1.0

    def _get_following_states(self, playboard, player):
        for i in available_actions(playboard):
            yield (i, self._copy_and_insert_into_playboard(column=i, playboard=playboard, player=player))    

    def _update(self, Q, gamma, learning_rate, state, action, againt_action, player):
        state_exe = self._copy_and_insert_into_playboard(state, action, player)
        state_encoded = encode_playboard(state)

        if state_encoded not in Q:
            Q[state_encoded] = np.full(state_exe.GetColumns(), 0.0)

        if state.GetWinner() == ConnectFourPlayboard.NONE:
            maxQ = None

            for i in self._get_following_states(state_exe, self._get_against_players(player)):
                encode_following_state = encode_playboard(i[1])

                if encode_following_state in Q:
                    if maxQ is None:
                        maxQ = Q[encode_following_state][againt_action]
                    else:
                        maxQ = max(maxQ, Q[encode_following_state][againt_action])
            
            Q[state_encoded][action] = (1.0 - learning_rate) * Q[state_encoded][action] + learning_rate * self._R(state_exe, player) + gamma * maxQ if maxQ is not None else 0.0
        else:
            Q[state_encoded][action] = (1.0 - learning_rate) * Q[state_encoded][action] + learning_rate * self._R(state_exe, player)
            
    def fit(self, againt_player_creator, gamma = 0.8, learning_rate = 0.01, iterations = 10 ** 5, useQ = False):
        winners = {ConnectFourPlayboard.WHITE: 0, ConnectFourPlayboard.RED: 0, ConnectFourPlayboard.NONE: 0}
        
        create_action = self._sample_next_action if not useQ else lambda playboard: self._next_action(playboard)[0]
        
        for i in range(iterations):
            playboard = ConnectFourPlayboard()
            against_player = againt_player_creator(playboard, self._get_against_players(self._player))
            assert against_player.GetPlayer() == self._get_against_players(self._player)
            
            history = []

            while playboard.GetWinner() == ConnectFourPlayboard.NONE and not playboard.IsFull():
                against_player.MakeMove()
                action = create_action(playboard)
                history.append((ConnectFourPlayboard(playboard), int(action)))
                playboard.Move(int(action), self._player)

            for j in range(len(history) - 1, -1, -1):
                #print('Update:', history[j][1], encode_playboard(history[j][0]), history[j+1][1] if (j + 1) < len(history) else -111111 )
                #show_playboard(history[j][0])
                self._update(Q=self._Q, gamma=gamma, learning_rate=learning_rate, action=history[j][1], againt_action=history[j+1][1] if (j + 1) < len(history) else None, player=self._player, state=history[j][0])


            display.clear_output(wait=True)
            winners[playboard.GetWinner()] += 1
            relativeWinners = lambda p: 100.0 * float(winners[p]) / float(np.sum(list(winners.values())))
            print('(%3d%%): Winner is Red: %5d (%3d%%), Winner is White: %5d (%3d%%), No winner: %5d (%3d%%)' % (float(i + 1) / float(iterations) * 100.0, winners[ConnectFourPlayboard.RED], relativeWinners(ConnectFourPlayboard.RED), winners[ConnectFourPlayboard.WHITE], relativeWinners(ConnectFourPlayboard.WHITE), winners[ConnectFourPlayboard.NONE], relativeWinners(ConnectFourPlayboard.NONE)))
                       

            show_playboard(playboard)

        for q in sorted(self._Q):
            print("%20d:\t%s" % (q, ','.join(['%5.4f' %i for i in self._Q[q]])))

## Training
1. Train with a random player. Use to Q matrix to get better results
2. Train with KI (Minimax algrithmn) without Q matrix (random actions) to finalize the Q matrix

In [5]:
class ConnectFourRandomPlayer(ConnectFourPlayerInterface):
    def __init__(self, playboard, player):
        # protected
        self._playboard = playboard
        self._player = player
        self._Q = {}

    # Impl interface
    def GetPlayer(self):
        return self._player
    
    def MakeMove(self):
        column = int(np.random.choice(available_actions(self._playboard)))
        self._playboard.Move(column, self._player)
        return column
        

In [6]:
player = ConnectFourQLearningPlayer(player=ConnectFourPlayboard.WHITE)

iterations = 10 ** 3
gamma = 0.8
learning_rate = 0.1

In [7]:
trainerRandom = lambda playboard, player: ConnectFourRandomPlayer(playboard, player)
player.fit(iterations=iterations, againt_player_creator=trainerRandom, useQ=True)

(100%): Winner is Red:   529 ( 52%), Winner is White:   471 ( 47%), No winner:     0 (  0%)


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


                   1:	-0.0026,2.4786,0.0000,0.0000,0.0000,0.0000,0.0000
                   3:	3.8658,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                   9:	3.8720,0.0000,0.0000,-0.0007,0.0000,0.0000,0.0000
                  14:	0.0000,0.0573,0.0000,0.0000,0.0000,0.0000,0.0000
                  16:	0.0986,-0.0005,0.0000,0.0000,0.0000,0.0000,0.0000
                  27:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2467
                  32:	0.2747,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  34:	-0.0004,-0.0009,0.0084,0.0000,0.0000,0.0000,-0.0017
                  38:	0.2735,0.0000,0.0000,0.0000,-0.0011,0.0000,0.0000
                  66:	-0.0009,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  81:	-0.0021,-0.0013,0.1568,-0.0002,0.0000,0.0000,0.0000
                  86:	0.1305,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  88:	0.0470,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  92:	0.1359,-0.0026,0.0000,0.0000,0.0000,0.0000,0

               14109:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               14181:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               14192:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               14210:	0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000
               14270:	0.0000,0.0000,0.0003,0.0000,0.0000,0.0000,0.0000
               14272:	0.0000,-0.0009,0.0000,0.0000,0.0000,0.0000,0.0000
               15323:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0064,0.0000
               15325:	-0.0002,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               15458:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               15668:	-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               15730:	0.0000,0.0000,0.0000,-0.0013,0.0000,0.0000,0.0000
               15836:	0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000
               16045:	-0.0033,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               16108:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000

             1615913:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0021
             1619210:	0.0000,0.0000,-0.0026,0.0000,0.0000,0.0000,0.0000
             1620125:	-0.0011,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1632275:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1632407:	0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000
             1634766:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1635004:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1639558:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1641084:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1646125:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1647887:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1647959:	0.0000,0.0000,0.0000,0.0000,0.0004,0.0000,0.0000
             1654559:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051,0.0000
             1654721:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 

...

   16807096037489921:	0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000
   16829740082189969:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
   16852615897895978:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   16852619199167864:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0100
   16876183269547967:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   16936668249380789:	0.0000,0.0000,0.0000,0.0000,0.0041,0.0000,0.0000
   16936669038570674:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0051,0.0000
   17104323375145672:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0064
   17105160565199758:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
   17107704755551021:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
   17302767678377045:	-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   17516767581380914:	0.0000,-0.0033,0.0000,0.0000,0.0000,0.0000,0.0000
   17928349547399291:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   17933527422234776:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000

   50077355273141875:	0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000
   50077371934072913:	0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000
   50077372020225404:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080
   50077378993794287:	0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000
   50077403414507698:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0041
   50077403513211878:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0011,0.0000
   50077403516583682:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50077403785280606:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50077405837741373:	0.0000,0.0000,0.0000,0.0000,-0.0013,0.0000,0.0000
   50077405855455523:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50077406614176674:	0.0000,0.0000,0.0000,0.0000,-0.0017,0.0000,0.0000
   50077410388253647:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051
   50077413853078163:	0.0000,-0.0051,0.0000,0.0000,0.0000,0.0000,0.0000
   50077417343386012:	0.0000,0.0000,0.0000,-0.0021,0.0000,0.0000,0.

 1352126347246544219:	0.0000,0.0000,0.0000,0.0000,-0.0051,0.0000,0.0000
 1355832419005307510:	0.0000,0.0000,0.0000,0.0000,-0.0064,0.0000,0.0000
 1357049077388875187:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1357049077776649970:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1357064629601572358:	-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1362598336510629008:	-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1371861655123055029:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
 1384841269105307999:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0064,0.0000
 1386694289295222722:	-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1386694290467050127:	0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000
 1402141789930613381:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1402141978217148182:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1402141978605631553:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 1402141986741461822:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000

12163227627856162760:	-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12163229604650749337:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12163229636060506760:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12163229798408658098:	0.0000,0.0000,-0.0051,0.0000,0.0000,0.0000,0.0000
12163230484123957181:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12163244860531728359:	0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000
12163244923293847586:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
12163288176797916767:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12163649206270332383:	0.0000,0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000
12163649771129411906:	-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12163650639338727755:	0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000
12164464577007309671:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12165083504312813396:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
12166975485294744868:	0.0041,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000

37412953052341287956:	0.0051,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
37457415409068496321:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0064,0.0000
37457415693822555736:	0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000
37457416541886006157:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
38108771283388679378:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
38741979275109371311:	0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000,0.0000
38743832358060342370:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
39208680284182477790:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
40610852330324123581:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
40610854872189965032:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
40610854872276117523:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
41146871886215261051:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0064,0.0000
41152436030513473232:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0080,0.0000
42796392594810975295:	0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000


In [8]:
trainerMinimax = lambda playboard, player: ConnectFourPlayerMinimaxAlphaBeta(playboard, player, 4)
player.fit(iterations=iterations, gamma=gamma, learning_rate=learning_rate, againt_player_creator=trainerMinimax)

(100%): Winner is Red:   997 ( 99%), Winner is White:     3 (  0%), No winner:     0 (  0%)


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


                   1:	-0.0026,2.4786,0.0000,0.0000,0.0000,0.0000,0.0000
                   3:	3.8658,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                   9:	3.8720,0.0000,0.0000,-0.0007,0.0000,0.0000,0.0000
                  14:	0.0000,0.0573,0.0000,0.0000,0.0000,0.0000,0.0000
                  16:	0.0986,-0.0005,0.0000,0.0000,0.0000,0.0000,0.0000
                  27:	0.3465,0.0113,0.0054,-7.4823,-5.2761,-10.5043,0.0022
                  32:	0.2747,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  34:	-0.0004,-0.0009,0.0084,0.0000,0.0000,0.0000,-0.0017
                  38:	0.2735,0.0000,0.0000,0.0000,-0.0011,0.0000,0.0000
                  66:	-0.0009,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  81:	-0.0021,-0.0013,0.1568,-0.0002,0.0000,0.0000,0.0000
                  86:	0.1305,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  88:	0.0470,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  92:	0.1359,-0.0026,0.0000,0.0000,0.0000,0.00

                5213:	0.0078,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                5228:	0.0033,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                5236:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                5351:	-0.0026,0.0000,0.0000,-0.0011,0.0000,0.0000,0.0000
                5353:	0.0005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                5357:	0.0000,0.0000,-0.0005,0.0000,0.0000,0.0000,0.0000
                5375:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0064
                5390:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                5398:	0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000,0.0000
                5414:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                5429:	0.0000,-0.0051,0.0000,0.0000,0.0000,0.0000,0.0000
                5444:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                5462:	0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000
                5474:	-0.1000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000

              103829:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
              104449:	0.0000,-0.1000,0.0000,-0.1900,0.0000,0.0000,0.0000
              111913:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
              118245:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0004,0.0000
              118451:	-0.1216,0.0000,-0.2168,-0.2168,-0.0800,-0.1520,0.0000
              118455:	-0.0720,-0.5052,-0.2751,-0.0800,-0.1520,-0.3116,-0.3996
              118466:	-0.1900,-0.1900,-0.1000,0.0000,0.0000,-0.1000,-0.1900
              118623:	0.0000,-0.0640,0.0000,-0.0800,-0.1520,0.0000,0.0000
              118786:	0.0000,0.0000,-0.1000,0.0000,0.0000,-0.1000,0.0000
              119198:	-0.1000,0.0000,-0.1000,0.0000,0.0000,0.0000,-0.1000
              119202:	-0.1000,0.0000,-0.1000,0.0000,0.0000,-0.1000,-0.1000
              119595:	-0.3471,-0.0720,-0.0800,-0.0037,-0.1520,0.0000,-0.0583
              119678:	-0.1000,-0.1000,0.0000,0.0000,-0.1900,-0.1000,0.0000
              119758:	0.0000,-0.100

...

       7632582265211:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
       7632582639251:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
       7632582639512:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
       7632584068025:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
       7632587005289:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0026
       7632596935591:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
       7632617310345:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0512
       7632659906216:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051
       7632703443377:	0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000
       7632708242000:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
       7633032225467:	0.0000,0.0000,0.0000,0.0000,0.0011,0.0000,0.0000
       7633046438978:	0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000
       7633357238363:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
       7633358307806:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 

      45764055422366:	0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000,0.0000
      45764055425057:	0.0000,0.0000,0.0000,0.0000,-0.0041,0.0000,0.0000
      45764055435497:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051
      45764055489931:	0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000
      45764056428527:	-0.0051,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
      45764056428800:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0041,0.0000
      45764056789430:	0.0000,0.0017,0.0000,0.0000,0.0000,0.0000,0.0000
      45764056966742:	0.0000,0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000
      45764057491628:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
      45764058027518:	0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000
      45764058027563:	-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
      45764058384025:	0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000
      45764069189966:	0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000,0.0000
      45764069190638:	-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000,0

     214183032760067:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     214276389163700:	0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000
     218816048583169:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     221242594686647:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     222183443169844:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     226326340692212:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     228977485832459:	-0.0026,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     228977977364953:	-0.0041,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     228984419945599:	0.0000,-0.0021,0.0000,0.0000,0.0000,0.0000,0.0000
     229002399893498:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     229008579693289:	-0.0021,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
     229015829703022:	0.0000,0.0000,0.0000,-0.0026,0.0000,0.0000,0.0000
     229720903466677:	0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000
     244130793899402:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000

   50369033757076681:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50489787846561793:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50496825335322445:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0041
   50497195037212025:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080
   50559642739046948:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50626885968319147:	0.0000,0.0000,0.0000,0.0100,0.0000,0.0000,0.0000
   50672684847755957:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50673529021443656:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50673591783563603:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50695266445822469:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50695266833242964:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   51290072171527850:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   51292901246056277:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   51300531375742148:	0.0000,0.0000,0.0000,0.0000,-0.0064,0.0000,0.0000
   

  405931629625706555:	0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000
  422600774330808628:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0080
  450489891220284553:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
  450489985352664109:	0.0000,0.0000,0.0000,0.0000,0.0033,0.0000,0.0000
  450489985367013178:	0.0041,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  450489985378173439:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0051,0.0000
  450489985750518686:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  450489988864958326:	0.0000,0.0000,0.0000,0.0064,0.0000,0.0000,0.0000
  450490459297703751:	0.0000,0.0000,0.0000,-0.1000,0.0000,0.0000,0.0000
  450491690788090775:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  450497614720723639:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0080,0.0000
  450497617969351678:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  450497619141179083:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  450497897151323002:	0.0000,0.0000,0.0000,0.0000,0.0100,0.0000,0.0000
  4

## Test

In [9]:
playboard = ConnectFourPlayboard()

player.playboard = playboard

players = [ConnectFourPlayerMinimax(playboard, ConnectFourPlayboard.RED, 1), player]

while playboard.GetWinner() == ConnectFourPlayboard.NONE and not playboard.IsFull():
    move = [0, 0]
    for i in range(len(players)):
        move[i] = players[i].MakeMove()
    
    print('Player0 = %d, Player1 = %d' % (move[0], move[1]))
    show_playboard(playboard)
    
print('The Winner is: %s' % 'Red' if playboard.GetWinner() == ConnectFourPlayboard.RED else ('White' if playboard.GetWinner() == ConnectFourPlayboard.WHITE else 'nobody'))
    

Player0 = 3, Player1 = 0


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


Player0 = 3, Player1 = 6


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


Player0 = 3, Player1 = 0


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


Player0 = 3, Player1 = 5


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


The Winner is: Red
